In [1]:
import numpy as np
import pandas as pd

def get_acc(predictions, df_truth):
    score = (predictions == predictions.max(axis=1, keepdims=True)).astype(float)
    score /= score.sum(axis=1, keepdims=True)
    return score[df_truth.item.values, df_truth.truth.values].sum() / df_truth.shape[0]

In [5]:
data_path = '../data/'
datasets = [
#     ('crowdscale2013/sentiment',   'senti'),
#     ('crowdscale2013/fact_eval',   'fact'),
    ('active-crowd-toolkit/CF',           'CF'),
    ('active-crowd-toolkit/CF_amt',       'CF_amt'),
    ('active-crowd-toolkit/MS',           'MS'),
    ('active-crowd-toolkit/SP',           'SP'),
    ('active-crowd-toolkit/SP_amt',       'SP_amt'),
    ('active-crowd-toolkit/ZenCrowd_all', 'ZC_all'),
    ('active-crowd-toolkit/ZenCrowd_in',  'ZC_in'),
    ('active-crowd-toolkit/ZenCrowd_us',  'ZC_us'),
#     ('crowd_truth_inference/d_Duck Identification',            'duck'),
    ('crowd_truth_inference/d_jn-product',                     'product'),
    ('crowd_truth_inference/d_sentiment',                      'senti_1k'),
#     ('crowd_truth_inference/s4_Dog data',                      'dog'),
    ('crowd_truth_inference/s4_Face Sentiment Identification', 'face'),
#     ('crowd_truth_inference/s4_Relevance',                     'rel'),
    ('crowd_truth_inference/s5_AdultContent',                  'adult'),
    ('SpectralMethodsMeetEM/bluebird', 'bird'),
    ('SpectralMethodsMeetEM/dog',      'dog'),
    ('SpectralMethodsMeetEM/rte',      'rte'),
    ('SpectralMethodsMeetEM/trec',     'trec'),
    ('SpectralMethodsMeetEM/web',      'web')
]

In [4]:
datasets = [
        ('rbm/mniste_cs_test_dataset','MnistE'),
        ('rbm/condind_test_dataset','CondInd'),
        ('rbm/hs3_test_dataset','HS3'),
        ('rbm/mixedclf_test_dataset','MixedClf'),
        ('rbm/nnt_test_dataset','NeuralNet'),
        ('rbm/c2_boosting_test_dataset','C-Boosting'),
        ('rbm/c2_complement_test_dataset','C-Complement'),
        
    ]

In [2]:
import os

def generate_dataset_list(folder_path):
    datasets = []
    for item in os.listdir(folder_path):
        if os.path.isdir(os.path.join(folder_path, item)) and item.endswith('_train_dataset'):
            # Remove '_train_dataset' suffix and create a readable name
            name = item[:-14]  # Remove '_train_dataset'
            readable_name = ''.join(word.capitalize() for word in name.split('-'))
            datasets.append((f'deem/{item}', readable_name))
    return datasets

# Usage:
# folder_path = 'path/to/your/deem/folder'
# datasets = generate_dataset_list(folder_path)


In [3]:
datasets = [
        ('deem/hs3_train_dataset','HS3'),
        ('deem/mniste_train_dataset','MnistE'),
        ('deem/petfinder_train_dataset','Petfinder'),

    ]
datasets = [
    ('deem/hs3_train_dataset', 'HS3'),
    ('deem/mniste_train_dataset', 'MnistE'),
    ('deem/petfinder_train_dataset', 'Petfinder'),
    ('deem/artificial-characters_train_dataset', 'ArtificialCharacters'),
    ('deem/csgo_train_dataset', 'CSGO'),
    ('deem/eye_movements_train_dataset', 'EyeMovements'),
    ('deem/GesturePhaseSegmentationProcessed_train_dataset', 'GesturePhaseSegmentation'),
    #('deem/mboosting_train_dataset', 'MBoosting'),
    #('deem/mcomplement_train_dataset', 'MComplement'),
    ('deem/microaggregation2_train_dataset', 'Microaggregation2'),
    ('deem/pendigits_train_dataset', 'Pendigits'),
    #('deem/volcanoes-b2_train_dataset', 'VolcanoesB2'),
    ('deem/tree3k_train_dataset','Tree3K')
]

#datasets = [
#    ('deem/tree3k_train_dataset','Tree3K')
#]

In [4]:
from ebcc import ebcc_vb
data_path = '../data/'

In [ ]:
%%time
records = []
for dataset, abbrev in datasets:
    df_label = pd.read_csv(data_path + dataset + '/label.csv')
    df_label = df_label.drop_duplicates(keep='first')
    
    elbos = []
    seeds = []
    results = []
    for _ in range(5):
        seed = np.random.randint(1e8)
        prediction, elbo = ebcc_vb(df_label.values, num_groups=10, seed=seed, empirical_prior=True)
        elbos.append(elbo)
        results.append((prediction, seed, elbo))
        
    prediction_ik, seed, elbo = results[np.argmax(elbos)]
        
    df_truth = pd.read_csv(data_path + dataset + '/truth.csv')
    records.append((abbrev, get_acc(prediction_ik, df_truth), seed, elbo))    
    print('%-10s %10g %10d %10g'%records[-1])
    
df = pd.DataFrame.from_records(records, columns=['dataset', 'accuracy', 'seed', 'elbo'])
print(df['accuracy'].mean())
# display(df)

In [12]:
results

40

This only takes one result with the highest elbo, then calculate the acc.
